# Predictive Maintenance `Data Preparation`

In [1]:
# Predictive Maintenance
from sklearn.linear_model import LogisticRegression
from rpy2 import robjects
import pandas as pd
import numpy as np

## rpy2

In [2]:
from rpy2.robjects import pandas2ri
from rpy2.robjects import default_converter
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.lib.dplyr import DataFrame

C:\Users\Kajornphat\Anaconda3\lib\site-packages\rpy2\robjects\lib\dplyr.py:24: UserWarning: This was designed againt dplyr version 0.7.5 but you have 0.8.3
  warnings.warn('This was designed againt dplyr version %s but you have %s' % (TARGET_VERSION, dplyr.__version__))


In [4]:
r = robjects.r

## Declare Function

In [ ]:
def convert_df(dataframe):      
    '''Convert between 
    Pandas dataframe to R "Dataframe"'''
    with localconverter(default_converter + pandas2ri.converter) as cv:
        try:
                dataframe = pandas2ri.ri2py(dataframe)
        except:
                dataframe = DataFrame(dataframe)
    return dataframe

In [ ]:
def round_time(df):
    df['t_stamp'] = pd.to_datetime(df['t_stamp'], format='%Y%m%d_%H%M%S', errors='coerce')
    df = df.sort_values(by=['t_stamp'])
    df['t_stamp'] = pd.Series(df['t_stamp']).dt.round('H')
    return pd.DataFrame(df)

In [ ]:
def check_nan(df):
    for i in df:
        # Count no. NaN in columns
        no_Nan = df[i].isna().sum()
        no_col = df[i].shape[0]
        percent = (no_Nan/no_col) * 100
        
        print("{:.2f}% Missing in Columns {}".format(percent, i))

### Data Preperation `Raw Ignition`

In [5]:
file_name = ["cement_mill", "roller_press", "separator", "feeder"]
df = {file:pd.read_csv('Data/' + file + '.csv') for file in file_name}

In [6]:
df1 = pd.merge(df["cement_mill"], df["roller_press"], on='t_stamp')
df2 = pd.merge(df["separator"], df["feeder"], on='t_stamp')
df = pd.merge(df1, df2, on='t_stamp')

### Data Preperation `Vibration`

In [7]:
df_vib = pd.read_csv('Data/vib_test.csv')

In [9]:
df_vib = round_time(df_vib)

In [13]:
check_nan(df_vib)

0.06% Missing in Columns t_stamp
1.37% Missing in Columns vib 1-1
1.37% Missing in Columns vib 1-2
1.37% Missing in Columns vib 1-3
1.37% Missing in Columns vib 1-4
1.73% Missing in Columns vib 2-1
1.73% Missing in Columns vib 2-2
1.73% Missing in Columns vib 2-3
1.73% Missing in Columns vib 2-4
1.31% Missing in Columns vib 3-1
1.31% Missing in Columns vib 3-2
1.31% Missing in Columns vib 3-3
1.31% Missing in Columns vib 3-4
1.31% Missing in Columns vib 4-1
1.31% Missing in Columns vib 4-2
1.31% Missing in Columns vib 4-3
1.31% Missing in Columns vib 4-4


#### Remove SV and MV Features

In [7]:
### Remove SV and MV Features
df = df[df.columns[~df.columns.str.endswith('sv')]]
df = df[df.columns[~df.columns.str.endswith('mv')]]

### R Scripts 
1. Replace 0 value as NA
2. Remove Columns with Low Variance
3. Label Target by Timestamp (1 Month before Breakdown)
4. Remove Non-operating Machine

In [8]:
r.assign('dataset', convert_df(df))
r('''

library(caret)
library(dplyr)

bad_cols <- nearZeroVar(dataset)
dataset <- dataset[, -bad_cols]

dataset[dataset == 0] <- NA

dataset["target"] <- ifelse(dataset$t_stamp >= "2017-12-07", 1, 0)

dataset <- dataset %>%
    filter(get("mill.drive_power") > 4000)

''')
df = convert_df(r('dataset'))

C:\Users\Kajornphat\Anaconda3\lib\site-packages\rpy2\robjects\pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


## Python 
1. Remove Entire Columns(> 10% missing)
2. Remove Entire Rows
3. Differential roller press gap and press

In [38]:
for i in df:
    # Count no. NaN in columns
    no_Nan = df[i].isna().sum()
    no_col = df[i].shape[0]
    percent = (no_Nan/no_col) * 100
    if percent >= 10:
        df = df.drop(columns=i)

In [40]:
# Remove row
df = df.dropna()

In [42]:
# Subtraction (Drive/ Non-Drive) 
gap = df["roller_drive_gap"] - df["roller_nondrive_gap"]
pres = df["roller_drive_pres"] - df["roller_nondrive_pres"]

del df["roller_drive_gap"], df["roller_nondrive_gap"]
del df["roller_drive_pres"], df["roller_nondrive_pres"]

gap = gap.to_frame("roller_gap_diff")
pres = pres.to_frame("roller_pres_diff")
df = df.join(gap)
df = df.join(pres)

In [43]:
df

,t_stamp,be.mill_out_amp,mill.drive_power,mill_ch1_water.spray,mill_in_pres,mill_out_pres,mill_out_temp,be.rp_in_amp,feed.gate1_open,feed.gate2_open,...,slide.gate_nondrive_open,sep.fan_damper,sep.fan_power,sep_motor_amp,sep_motor_speed,hopper_cli_lev,tot_feed,target,roller_gap_diff,roller_pres_diff
1000,2017-07-11 17:30:00.000,251.923584,4273.349121,0.053075,-28.935339,-250.923981,126.713966,130.178650,47.381687,99.786583,...,97.120712,99.696135,463.722473,390.477783,952.968750,13.299668,280.599609,0.0,-0.486372,-6.755157
1001,2017-07-11 17:40:00.000,255.334427,4277.801758,0.053075,-29.130856,-251.989990,126.870308,126.893562,47.383213,99.799301,...,97.117760,99.696493,464.878174,396.738403,953.745789,13.922619,282.137604,0.0,-1.247070,-2.915878
1002,2017-07-11 17:50:00.000,247.377350,4277.956543,0.053076,-29.906759,-247.422348,126.931992,120.473251,47.389839,99.761177,...,97.127960,99.696777,463.244202,397.147644,951.633606,14.342266,272.556641,0.0,3.422939,3.200470
1003,2017-07-11 18:00:00.000,255.961304,4267.671387,0.053076,-29.725098,-249.113831,126.949463,127.984856,47.440613,99.781258,...,97.127409,99.696636,464.672089,388.905090,949.706238,13.271332,279.856201,0.0,1.654068,5.646271
1004,2017-07-11 18:10:00.000,253.275940,4268.598633,0.053077,-29.147459,-250.674210,126.850761,127.314163,47.392677,99.761108,...,97.127072,99.696519,464.270813,386.950165,949.801941,12.757819,280.653076,0.0,0.598497,1.676758
1005,2017-07-11 18:20:00.000,255.209778,4297.000000,0.053078,-27.881514,-251.186859,126.768188,126.771492,47.416573,99.795197,...,97.126900,99.696402,464.648102,380.818604,949.973511,13.687346,279.495728,0.0,0.845537,2.815048
1006,2017-07-11 18:30:00.000,254.747086,4274.054199,0.053078,-29.018036,-249.548035,126.688538,128.941010,47.411743,99.794403,...,97.128029,99.696285,464.299255,383.766968,951.857422,13.103842,280.659668,0.0,1.134829,5.368011
1007,2017-07-11 18:40:00.000,250.900787,4327.030762,0.053079,-30.464561,-250.976013,126.568909,129.271927,47.467876,99.800644,...,97.123322,99.696168,462.498108,392.806702,949.849304,13.383018,273.362640,0.0,1.648710,-1.896957
1008,2017-07-11 18:50:00.000,256.299255,4288.367188,0.053080,-28.342819,-249.502960,126.555794,125.252670,47.470947,99.774834,...,97.122375,99.696051,463.377167,387.888489,951.893555,12.484408,277.161346,0.0,-0.458134,-1.416382
1009,2017-07-11 19:00:00.000,254.026535,4317.984375,0.053080,-29.975780,-247.681503,126.523796,124.477951,47.385113,99.794571,...,97.119576,99.695934,466.324554,373.974274,940.433533,13.974775,277.765350,0.0,0.605024,-7.185760
